In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, Embedding
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

import matplotlib.pyplot as plt

import keras_tuner as kt

import shap


c:\Users\Gustavo\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Gustavo\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Gustavo\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/f

In [2]:
# Conectar ao banco de dados SQLite
DB_FILE = "dados_acoes.db"
conn = sqlite3.connect(DB_FILE)

# Carregar as tabelas
# O usuário confirmou que as colunas 'date' já estão em formato datetime
try:
    df_diario = pd.read_sql_query("SELECT * FROM diario_com_indicadores", conn)
    df_semanal = pd.read_sql_query("SELECT * FROM semanal_com_indicadores", conn)
    df_mensal = pd.read_sql_query("SELECT * FROM mensal_com_indicadores", conn)
    
    
    
    # Verificação extra: garantir que o pandas as reconheceu como datetime
    # Se elas forem strings, o merge_asof falhará.
    df_diario['datetime'] = pd.to_datetime(df_diario['datetime'])
    df_semanal['datetime'] = pd.to_datetime(df_semanal['datetime'])
    df_mensal['datetime'] = pd.to_datetime(df_mensal['datetime'])

    print(f"Diário: {df_diario.shape}")
    print(f"Semanal: {df_semanal.shape}")
    print(f"Mensal: {df_mensal.shape}")
    
    print("\nTipo da coluna 'datetime' (Diário):", df_diario['datetime'].dtype)

except Exception as e:
    print(f"Erro ao carregar dados: {e}")
    print("Verifique se o nome do arquivo 'dados_acoes.db' e os nomes das tabelas estão corretos.")

finally:
    conn.close()

# Visualizar os dados
print("\nAmostra Dados Diários:")
print(df_diario.head())

Diário: (411144, 29)
Semanal: (85971, 29)
Mensal: (19802, 29)

Tipo da coluna 'datetime' (Diário): datetime64[ns]

Amostra Dados Diários:
    datetime     close      high       low      open  volume    ticker  EMA_9  \
0 2000-01-05  0.248809  0.248809  0.248809  0.248809     985  ABEV3.SA    NaN   
1 2000-01-06  0.236196  0.236196  0.236196  0.236196     227  ABEV3.SA    NaN   
2 2000-01-07  0.236196  0.236196  0.236196  0.236196     151  ABEV3.SA    NaN   
3 2000-01-10  0.236196  0.236196  0.236196  0.236196    1516  ABEV3.SA    NaN   
4 2000-01-11  0.236196  0.236196  0.236196  0.236196    3791  ABEV3.SA    NaN   

   SMA_21  SMA_50  ...  BBP_20_2.0_2.0  STOCHk_14_3_3  STOCHd_14_3_3  \
0     NaN     NaN  ...             NaN            NaN            NaN   
1     NaN     NaN  ...             NaN            NaN            NaN   
2     NaN     NaN  ...             NaN            NaN            NaN   
3     NaN     NaN  ...             NaN            NaN            NaN   
4     NaN     N

In [3]:
# Garantir que tudo está ordenado por data para o merge_asof funcionar
df_diario = df_diario.sort_values(by='datetime')
df_semanal = df_semanal.sort_values(by='datetime')
df_mensal = df_mensal.sort_values(by='datetime')

# Renomear colunas de indicadores para evitar conflitos (ex: 'RSI' diário, 'RSI' semanal)
df_semanal = df_semanal.add_suffix('_sem')
df_mensal = df_mensal.add_suffix('_men')

# Renomear colunas de junção
df_semanal = df_semanal.rename(columns={'datetime_sem': 'datetime', 'ticker_sem': 'ticker'})
df_mensal = df_mensal.rename(columns={'datetime_men': 'datetime', 'ticker_men': 'ticker'})

# ---------------------------------------------------------------------------------
# Junção (Merge)
# ---------------------------------------------------------------------------------
# 1. Juntar Diário com Semanal
# Para cada 'ticker', vamos juntar a data diária com a data semanal mais próxima (anterior ou igual)
df_merged = pd.merge_asof(
    df_diario,
    df_semanal,
    on='datetime',
    by='ticker',
    direction='backward' # 'backward' pega o último dado semanal disponível para aquele dia
)

# 2. Juntar o resultado com o Mensal
df_merged = pd.merge_asof(
    df_merged,
    df_mensal,
    on='datetime',
    by='ticker',
    direction='backward'
)

print("\nAmostra de Dados Unificados:")
print(df_merged.head())

print(f"\nShape após merge: {df_merged.shape}")


Amostra de Dados Unificados:
    datetime       close        high         low        open    volume  \
0 2000-01-03    0.444428    0.463940    0.444428    0.455268   1029600   
1 2000-01-03   53.865044   53.865044   53.865044   53.865044       145   
2 2000-01-03    0.327688    0.345801    0.306281    0.307928  13152318   
3 2000-01-03  129.651276  131.126621  128.161031  129.651276    150000   
4 2000-01-03  214.932358  217.591315  212.716560  213.159720   1210000   

     ticker      EMA_9     SMA_21     SMA_50  ...  BBP_20_2.0_2.0_men  \
0  BBAS3.SA  10.141362  12.038306  12.614476  ...           -0.404727   
1  LIGT3.SA  24.982800  19.414526  18.789501  ...            1.552200   
2  ITSA4.SA  38.178534  45.309890  46.331954  ...           -0.385870   
3  GOAU4.SA  30.972806  12.267680   9.200727  ...            1.561928   
4  GGBR4.SA  54.807740  24.783922  19.350447  ...            1.561622   

   STOCHk_14_3_3_men  STOCHd_14_3_3_men  STOCHh_14_3_3_men       OBV_men  \
0         

In [4]:
# --- Definição do Alvo (y) ---
PERIOD_HORIZON = 15
TIME_STEPS = 30 # Hiperparâmetro: quantos dias o LSTM vai "olhar para trás"

# <-- MUDANÇA: Threshold para classificação binária (0 = retorno > 0%)
CLASSIFICATION_THRESHOLD = 0.05

# THRESHOLD
THRESHOLD = 0.7

# <-- MUDANÇA: Novo nome de arquivo para o modelo de classificação
MODEL_FILE = "transformers_stock_model_30d_ts30_CLASS_weighted.keras" # Nome do arquivo para salvar/carregar

In [5]:
# Agrupar por ticker para calcular o shift corretamente
df_merged['close_future'] = df_merged.groupby('ticker')['close'].shift(-PERIOD_HORIZON)


# 1. Calcular o retorno futuro (para referência e para criar o alvo)
df_merged['y_return'] = (df_merged['close_future'] / df_merged['close']) - 1

# 2. Criar o alvo de CLASSIFICAÇÃO (1 se o retorno > threshold, 0 caso contrário)
df_merged['y_target'] = (df_merged['y_return'] > CLASSIFICATION_THRESHOLD).astype(int)

# --- Limpeza ---
# Remover dados onde não pudemos calcular o alvo (os últimos N dias de cada ticker)
# Também remover quaisquer NaNs gerados pelos merges ou cálculos de indicadores
df_final = df_merged.dropna()

if df_final.empty:
    print("ERRO: O DataFrame final está vazio após o dropna().")
    print("Verifique seus dados de entrada, a lógica de merge e o cálculo do 'y_target'.")
else:
    print(f"\nShape final após limpeza e 'y_target': {df_final.shape}")
    print(f"Distribuição do Alvo (y_target):\n{df_final['y_target'].value_counts(normalize=True)}")

    # --- Definição das Features (X) ---
    features_diarias = [col for col in df_final.columns if col not in ['datetime', 'ticker'] and not col.endswith('_sem') and not col.endswith('_men')]
    features_semanais = [col for col in df_final.columns if col.endswith('_sem') and col not in ['date_sem', 'ticker_sem']]
    features_mensais = [col for col in df_final.columns if col.endswith('_men') and col not in ['date_men', 'ticker_men']]
    all_features = features_diarias + features_semanais + features_mensais
    
    # Defina as colunas que NUNCA devem ser features
    # <-- MUDANÇA: Adicionado 'y_return' à exclusão
    colunas_a_excluir = ['datetime', 'ticker', 'close_future', 'y_return', 'y_target']
    
    # Filtra para garantir que são numéricas E não são as colunas de exclusão
    features_numericas = [col for col in all_features if pd.api.types.is_numeric_dtype(df_final[col])]
    features = [col for col in features_numericas if col not in colunas_a_excluir]
    print(f"\nUsando {len(features)} features:")
    print(features)


    print("\n--- Verificando Vazamento de Dados (Leakage) ---")
    leaky_cols = [col for col in features if col in ['close_future', 'y_return', 'y_target']]

    if len(leaky_cols) > 0:
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"!!! ALERTA DE VAZAMENTO (LEAKAGE) DETECTADO !!!")
        print(f"As seguintes colunas-alvo ESTÃO na sua lista 'features' (X): {leaky_cols}")
        print(f"Corrija sua lista 'colunas_a_excluir' na Célula [5].")
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
    else:
        print(">>> Verificação de leakage OK. Nenhuma coluna-alvo encontrada em X.\n")
    # -----------------------------------------------

    X = df_final[features]
    y = df_final['y_target']


Shape final após limpeza e 'y_target': (397298, 86)
Distribuição do Alvo (y_target):
y_target
0    0.711564
1    0.288436
Name: proportion, dtype: float64

Usando 81 features:
['close', 'high', 'low', 'open', 'volume', 'EMA_9', 'SMA_21', 'SMA_50', 'SMA_200', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0_2.0', 'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'STOCHh_14_3_3', 'OBV', 'ATRr_14', 'ADX_14', 'ADXR_14_2', 'DMP_14', 'DMN_14', 'close_sem', 'high_sem', 'low_sem', 'open_sem', 'volume_sem', 'EMA_9_sem', 'SMA_21_sem', 'SMA_50_sem', 'SMA_200_sem', 'RSI_14_sem', 'MACD_12_26_9_sem', 'MACDh_12_26_9_sem', 'MACDs_12_26_9_sem', 'BBL_20_2.0_2.0_sem', 'BBM_20_2.0_2.0_sem', 'BBU_20_2.0_2.0_sem', 'BBB_20_2.0_2.0_sem', 'BBP_20_2.0_2.0_sem', 'STOCHk_14_3_3_sem', 'STOCHd_14_3_3_sem', 'STOCHh_14_3_3_sem', 'OBV_sem', 'ATRr_14_sem', 'ADX_14_sem', 'ADXR_14_2_sem', 'DMP_14_sem', 'DMN_14_sem', 'close_men', 'high_men', '

In [6]:
# <-- MUDANÇA: Solução 2 - Split Treino/Validação/Teste Temporal

if not df_final.empty:
    # 1. Separar Teste (20% finais)
    split_test = int(len(df_final) * 0.8)
    df_train_val = df_final.iloc[:split_test] # 80% para treino+validação
    df_test = df_final.iloc[split_test:]      # 20% para teste final

    # 2. Separar Treino e Validação (dos 80% iniciais)
    split_val = int(len(df_train_val) * 0.8)
    df_train = df_train_val.iloc[:split_val] # 80% do df_train_val
    df_validation = df_train_val.iloc[split_val:] # 20% do df_train_val
    
    # 3. Separar X, y, e tickers para cada set
    X_train = df_train[features]
    y_train = df_train['y_target']
    ticker_train = df_train['ticker']
    
    X_validation = df_validation[features]
    y_validation = df_validation['y_target']
    ticker_validation = df_validation['ticker']
    
    X_test = df_test[features]
    y_test = df_test['y_target']
    ticker_test = df_test['ticker']
    
    print(f"Shape Treino 2D (X_train): {X_train.shape}")
    print(f"Shape Validação 2D (X_validation): {X_validation.shape}")
    print(f"Shape Teste 2D (X_test): {X_test.shape}")
    print(f"Datas de Treino: {df_train['datetime'].min()} a {df_train['datetime'].max()}")
    print(f"Datas de Validação: {df_validation['datetime'].min()} a {df_validation['datetime'].max()}")
    print(f"Datas de Teste: {df_test['datetime'].min()} a {df_test['datetime'].max()}")

    # --- Normalização (Scaling) CORRETA ---
    # 4. Instanciar o Scaler
    scaler = StandardScaler()
    
    # 5. Fit (Ajustar) o scaler APENAS nos dados de TREINO (X_train)
    scaler.fit(X_train)
    
    # 6. Transformar (Aplicar) em TODOS os três sets
    X_train_scaled_array = scaler.transform(X_train)
    X_val_scaled_array = scaler.transform(X_validation)
    X_test_scaled_array = scaler.transform(X_test)
    
    # 7. Criar DataFrames com os dados escalados (para a função de sequência)
    X_train_scaled = pd.DataFrame(X_train_scaled_array, columns=features, index=X_train.index)
    X_val_scaled = pd.DataFrame(X_val_scaled_array, columns=features, index=X_validation.index)
    X_test_scaled = pd.DataFrame(X_test_scaled_array, columns=features, index=X_test.index)
    
    print(f"\nX_train_scaled shape: {X_train_scaled.shape}")
    print(f"X_val_scaled shape: {X_val_scaled.shape}")
    print(f"X_test_scaled shape: {X_test_scaled.shape}")

else:
    print("DataFrame vazio, pulando etapas.")

Shape Treino 2D (X_train): (254270, 81)
Shape Validação 2D (X_validation): (63568, 81)
Shape Teste 2D (X_test): (79460, 81)
Datas de Treino: 2000-01-03 00:00:00 a 2020-01-06 00:00:00
Datas de Validação: 2020-01-06 00:00:00 a 2022-09-23 00:00:00
Datas de Teste: 2022-09-23 00:00:00 a 2025-10-07 00:00:00

X_train_scaled shape: (254270, 81)
X_val_scaled shape: (63568, 81)
X_test_scaled shape: (79460, 81)


In [7]:
def create_sequences_por_ticker(X_data, y_data, tickers, time_steps):
    """
    Cria sequências de dados 3D (para LSTM) agrupadas por ticker.
    Garante que as sequências não cruzem tickers diferentes.
    """
    all_X_seq, all_y_seq, all_indices = [], [], []
    
    # Usamos os índices originais de df_final para rastrear datas/tickers
    unique_tickers = tickers.unique()
    
    for i, ticker in enumerate(unique_tickers):
        # Filtrar dados para este ticker específico
        ticker_mask = (tickers == ticker)
        X_ticker = X_data[ticker_mask]
        y_ticker = y_data[ticker_mask]
        
        # O índice original é mantido
        ticker_indices = y_ticker.index
        
        # print(f"Processando Ticker: {ticker} ({i+1}/{len(unique_tickers)}) - {len(X_ticker)} amostras") # Debug
        
        # Aplicar a janela deslizante (sliding window) apenas neste ticker
        # Se o ticker tem menos dados que 'time_steps', ele será ignorado
        for j in range(len(X_ticker) - time_steps):
            # Sequência de features (ex: dias 0 a 29)
            seq = X_ticker.iloc[j:(j + time_steps)].values
            
            # Alvo (ex: dia 30)
            target = y_ticker.iloc[j + time_steps]
            
            # Índice do alvo (para rastrear data/ticker depois)
            target_index = ticker_indices[j + time_steps]
            
            all_X_seq.append(seq)
            all_y_seq.append(target)
            all_indices.append(target_index)
            
    return np.array(all_X_seq), np.array(all_y_seq), np.array(all_indices)

# <-- MUDANÇA: Solução 2 - Criar Sequências para os 3 sets
if 'X_train_scaled' in locals():
    
    print("Criando sequências de TREINO...")
    X_train_seq, y_train_seq, seq_indices_train = create_sequences_por_ticker(
        X_train_scaled, 
        y_train, 
        ticker_train, 
        TIME_STEPS
    )
    
    print("Criando sequências de VALIDAÇÃO...")
    X_val_seq, y_val_seq, seq_indices_val = create_sequences_por_ticker(
        X_val_scaled, 
        y_validation, 
        ticker_validation, 
        TIME_STEPS
    )
    
    print("Criando sequências de TESTE...")
    X_test_seq, y_test_seq, seq_indices_test = create_sequences_por_ticker(
        X_test_scaled, 
        y_test, 
        ticker_test, 
        TIME_STEPS
    )
    
    # Esta é a variável que a Célula [9] (Avaliação) usa
    indices_test = seq_indices_test
    
    print(f"\nFormato das Sequências de Treino (X): {X_train_seq.shape}")
    print(f"Formato dos Alvos de Treino (y): {y_train_seq.shape}")

    print(f"\nFormato das Sequências de Validação (X): {X_val_seq.shape}")
    print(f"Formato dos Alvos de Validação (y): {y_val_seq.shape}")
    
    print(f"\nFormato das Sequências de Teste (X): {X_test_seq.shape}")
    print(f"Formato dos Alvos de Teste (y): {y_test_seq.shape}")
else:
    print("Dados de treino/teste não encontrados. Rode as células anteriores.")

Criando sequências de TREINO...
Criando sequências de VALIDAÇÃO...
Criando sequências de TESTE...

Formato das Sequências de Treino (X): (251907, 30, 81)
Formato dos Alvos de Treino (y): (251907,)

Formato das Sequências de Validação (X): (60458, 30, 81)
Formato dos Alvos de Validação (y): (60458,)

Formato das Sequências de Teste (X): (76220, 30, 81)
Formato dos Alvos de Teste (y): (76220,)


In [ ]:
# --- Helper Function: Bloco Transformer Encoder ---
def transformer_encoder_block(inputs, d_model, head_size, num_heads, ff_dim, dropout_rate=0.1):
    """
    Cria um único bloco Transformer Encoder.
    d_model = dimensão da feature (no nosso caso, n_features)
    """
    
    # --- 1. Multi-Head Attention (Self-Attention) ---
    # O modelo "presta atenção" a diferentes partes da sequência de 30 dias
    attn_output = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs) # Query=inputs, Key=inputs, Value=inputs (self-attention)
    
    # Conexão Residual e Normalização
    attn_output = layers.Dropout(dropout_rate)(attn_output)
    out1 = layers.LayerNormalization(epsilon=1e-6)(inputs + attn_output)

    # --- 2. Feed Forward Network ---
    # Uma rede neural simples aplicada a cada "dia" (time step)
    ffn_output = layers.Dense(ff_dim, activation="relu")(out1)
    ffn_output = layers.Dense(d_model)(ffn_output) # Projeta de volta para a dimensão original
    
    # Conexão Residual e Normalização
    ffn_output = layers.Dropout(dropout_rate)(ffn_output)
    out2 = layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)
    
    return out2


# --- Função Construtora de Modelo (para KerasTuner) ---
def build_model(hp):
    """
    Função construtora de modelo para o KerasTuner, usando Transformers.
    """
    
    # Pegar as dimensões dos dados de treino
    n_samples, time_steps, n_features = X_train_seq.shape
    d_model = n_features # Dimensão do modelo é o número de features
    
    # --- Input ---
    inputs = layers.Input(shape=(time_steps, d_model), name="Input_Sequence")
    x = inputs
    
    # --- 1. Positional Embedding ---
    # O Transformer puro não sabe a *ordem* dos dias (é permutation-invariant).
    # Precisamos adicionar uma "Positional Embedding" para que ele saiba 
    # qual dia veio antes de qual. Usamos uma Embedding "aprendível".
    
    # Cria uma camada de embedding para as posições (0, 1, ..., 29)
    pos_embedding_layer = layers.Embedding(input_dim=time_steps, output_dim=d_model, name="PositionalEmbedding")
    # Cria as posições (constante)
    positions = tf.range(start=0, limit=time_steps, delta=1)
    # Adiciona o embedding da posição aos dados de entrada
    x = x + pos_embedding_layer(positions)
    
    
    # --- Hiperparâmetros para Tunar ---
    
    # 1. Número de blocos Transformer para empilhar
    hp_num_blocks = hp.Int('num_blocks', min_value=1, max_value=3, step=1)
    
    # 2. Parâmetros da Multi-Head Attention
    hp_num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2) # Ex: 2, 4, 8 cabeças
    hp_head_size = hp.Int('head_size', min_value=32, max_value=128, step=32) # key_dim
    
    # 3. Dimensão da camada Feed-Forward interna
    hp_ff_dim = hp.Int('ff_dim', min_value=d_model, max_value=d_model * 4, step=d_model) # Ex: 128, 256
    
    # 4. Dropout (para regularização)
    hp_dropout = hp.Float('dropout', min_value=0.1, max_value=0.3, step=0.1)
    
    # 5. Learning Rate
    hp_learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=5e-4, sampling='log')
    
    # 6. Unidades da camada Densa (igual ao que você tinha)
    hp_dense_units = hp.Int('dense_units', min_value=32, max_value=64, step=16)
    # ------------------------------------

    # --- 2. Stack de Encoders ---
    # Constrói a arquitetura empilhando os blocos
    for _ in range(hp_num_blocks):
        x = transformer_encoder_block(
            inputs=x,
            d_model=d_model,
            head_size=hp_head_size,
            num_heads=hp_num_heads,
            ff_dim=hp_ff_dim,
            dropout_rate=hp_dropout
        )

    # --- 3. Cabeça de Classificação (Classification Head) ---
    
    # A saída 'x' ainda é uma sequência (Batch, 30, 81).
    # Precisamos agregar tudo em um único vetor por amostra.
    # GlobalAveragePooling1D tira a média dos 30 time steps.
    x = layers.GlobalAveragePooling1D(name="Global_Pooling")(x)
    
    # Camada Densa final para classificação
    x = layers.Dropout(hp_dropout)(x)
    x = layers.Dense(units=hp_dense_units, activation='relu', name="Dense_Classifier")(x)
    
    # Camada final de classificação (sigmoid)
    outputs = layers.Dense(1, activation='sigmoid', name="Output")(x)

    # --- 4. Compilar o Modelo ---
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="Transformer_Tuner_Model")
    
    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='binary_crossentropy',
        metrics=[AUC(name='roc_auc', curve='ROC'), 'accuracy']
    )
    
    return model

# --- Configuração do KerasTuner ---


# Verificar se y_train_seq existe antes de calcular os pesos
if 'y_train_seq' in locals() and y_train_seq.size > 0:
    # Obter as classes únicas (ex: [0, 1])
    classes = np.unique(y_train_seq)
    
    # Calcular os pesos no modo 'balanced'
    # Isso atribui pesos maiores às classes menos frequentes
    weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=y_train_seq
    )
    
    # Criar o dicionário de pesos que o Keras espera
    # Ex: {0: 0.89, 1: 1.14}
    class_weight_dict = dict(zip(classes, weights))
    
    print(f"Pesos de Classe Calculados: {class_weight_dict}")
else:
    print("y_train_seq não encontrado. Pulando cálculo de pesos.")
    class_weight_dict = None # Definir como None se os dados não estiverem prontos


if 'X_train_seq' in locals():
    # Instanciar o Tuner. 
    tuner = kt.Hyperband(
        build_model,
        objective=kt.Objective("val_roc_auc", direction="max"),
        max_epochs=50,
        factor=3,
        directory='keras_tuner_dir',
        project_name='stock_TRANSFORMER_tuning_weighted_rocauc'
    )

    # Callback de EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    print("--- INICIANDO BUSCA POR HIPERPARÂMETROS (TRANSFORMER) ---")
    
    # Iniciar a busca
    tuner.search(
        X_train_seq, y_train_seq,
        epochs=50,
        batch_size=64,
        validation_data=(X_val_seq, y_val_seq),
        callbacks=[early_stopping],
        verbose=1,
        class_weight=class_weight_dict
    )

    print("--- BUSCA CONCLUÍDA ---")

    # 1. Pegar os melhores hiperparâmetros
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"""
    Melhores hiperparâmetros encontrados:
    - num_blocks: {best_hps.get('num_blocks')}
    - num_heads: {best_hps.get('num_heads')}
    - head_size: {best_hps.get('head_size')}
    - ff_dim: {best_hps.get('ff_dim')}
    - dropout: {best_hps.get('dropout'):.2f}
    - dense_units: {best_hps.get('dense_units')}
    - learning_rate: {best_hps.get('learning_rate'):.5f}
    """)

    # -----------------------------------------------------------------
    # SOLUÇÃO: RE-TREINAR O MELHOR MODELO
    # -----------------------------------------------------------------
    print("--- TREINANDO O MODELO FINAL COM OS MELHORES HPs ---")

    # 1. Construir o modelo final usando os melhores HPs
    #    (best_hps já foi pego na etapa anterior)
    model_transformer = build_model(best_hps)

    # 2. (Opcional, mas recomendado) Recupere o número ideal de epochs
    #    se o EarlyStopping parou o melhor "trial" mais cedo.
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
    print("Buscando a melhor época do 'best_trial'...")

    # 1. Pegar o histórico da métrica que seu EarlyStopping monitora (val_loss)
    #    Isso retorna uma lista de objetos: [MetricHistory(epoch=0, value=0.5), MetricHistory(epoch=1, value=0.45), ...]
    val_loss_history_objects = best_trial.metrics.get_history("val_loss")

    # 2. Extrair APENAS os valores numéricos para o numpy
    #    Isso cria uma lista como: [0.5, 0.45, 0.48]
    val_loss_values = [m.value for m in val_loss_history_objects]

    # 3. Encontrar o índice (época) que teve o *menor* valor de loss
    #    Usamos np.argmin() porque "loss" deve ser minimizada.
    best_epoch_index = np.argmin(val_loss_values)

    # 4. Converter o índice (base-0) para o número de épocas (base-1)
    #    Se o índice 0 for o melhor, treinamos por 1 época.
    #    Se o índice 1 for o melhor, treinamos por 2 épocas.
    best_epochs = best_epoch_index + 8

    print(f"Melhor época encontrada (baseada no val_loss): {best_epochs}")

    print(f"Treinando o modelo final por {best_epochs} epochs.")

    # 3. Treinar o modelo final. 
    #    Desta vez, usamos os mesmos dados de treino E validação.
    #    O EarlyStopping é usado para garantir que ele pare no ponto certo.
    history = model_transformer.fit(
        X_train_seq, y_train_seq,
        epochs=best_epochs, # Treina pelo número ideal de épocas
        batch_size=64,
        validation_data=(X_val_seq, y_val_seq),
        callbacks=[early_stopping], # Use o mesmo EarlyStopping
        class_weight=class_weight_dict,
        verbose=1
    )

    print("Modelo final (Transformer) treinado e carregado na variável 'model_transformer'.")
    model_transformer.summary()

    # 4. Salvar o modelo final treinado
    print(f"Salvando o melhor modelo em '{MODEL_FILE}'...")
    model_transformer.save(MODEL_FILE)
    print("Modelo salvo com sucesso.")
    # -----------------------------------------------------------------

else:
    print("AVISO: 'X_train_seq' não foi definido. Pulando hyperparameter tuning.")

Pesos de Classe Calculados: {np.int64(0): np.float64(0.704847898106281), np.int64(1): np.float64(1.7204176967941975)}
Reloading Tuner from keras_tuner_dir\stock_TRANSFORMER_tuning_weighted_rocauc\tuner0.json
--- INICIANDO BUSCA POR HIPERPARÂMETROS (TRANSFORMER) ---
--- BUSCA CONCLUÍDA ---

    Melhores hiperparâmetros encontrados:
    - num_blocks: 1
    - num_heads: 4
    - head_size: 64
    - ff_dim: 324
    - dropout: 0.20
    - dense_units: 32
    - learning_rate: 0.00202
    
--- TREINANDO O MODELO FINAL COM OS MELHORES HPs ---
Buscando a melhor época do 'best_trial'...
Melhor época encontrada (baseada no val_loss): 1
Treinando o modelo final por 1 epochs.
 276/3937 ━━━━━━━━━━━━━━━━━━━━ 52s 14ms/step - accuracy: 0.4950 - loss: 0.7018 - roc_auc: 0.5372

KeyboardInterrupt: 

In [ ]:
# <-- MUDANÇA: Solução 1 - Avaliação de Classificação

if 'model_transformer' in locals():
    # 1. Fazer previsões no set de teste (retorna probabilidades)
    y_pred_proba = model_transformer.predict(X_test_seq).flatten()
    
    # 2. Converter probabilidades em classes (ex: 0.5 como threshold)
    y_pred_class = (y_pred_proba > THRESHOLD).astype(int)

    # 3. Buscar as informações originais (data, ticker) usando os índices salvos
    test_info = df_final.loc[indices_test]

    # 4. Criar o DataFrame de resultados
    df_results = pd.DataFrame({
        'date': test_info['datetime'],
        'ticker': test_info['ticker'],
        'y_real': y_test_seq,        # Alvos reais (0 ou 1)
        'y_pred_proba': y_pred_proba,  # Probabilidade prevista (ex: 0.75)
        'y_pred_class': y_pred_class   # Classe prevista (0 ou 1)
    })

    # 5. Avaliação de Métricas de Classificação
    accuracy = accuracy_score(df_results['y_real'], df_results['y_pred_class'])
    print(f"\nAcurácia (Accuracy) no Teste: {accuracy * 100:.2f}%")
    
    # --- CÁLCULO DE ROC AUC ADICIONADO ---
    # (Usa as probabilidades, não as classes)
    roc_auc = roc_auc_score(df_results['y_real'], df_results['y_pred_proba'])
    print(f"ROC AUC no Teste: {roc_auc:.4f}\n")

    print("\nMatriz de Confusão:")
    # (Linhas = Real, Colunas = Previsto)
    print(confusion_matrix(df_results['y_real'], df_results['y_pred_class']))
    
    print("\nRelatório de Classificação:")
    print(classification_report(df_results['y_real'], df_results['y_pred_class'], target_names=['BAIXA (0)', 'ALTA (1)']))

    print("\nAmostra dos Resultados (Transformer):")
    print(df_results.head())

2382/2382 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step

Acurácia (Accuracy) no Teste: 73.88%
ROC AUC no Teste: 0.7301


Matriz de Confusão:
[[47344  8050]
 [11861  8965]]

Relatório de Classificação:
              precision    recall  f1-score   support

   BAIXA (0)       0.80      0.85      0.83     55394
    ALTA (1)       0.53      0.43      0.47     20826

    accuracy                           0.74     76220
   macro avg       0.66      0.64      0.65     76220
weighted avg       0.73      0.74      0.73     76220


Amostra dos Resultados (Transformer):
             date    ticker  y_real  y_pred_proba  y_pred_class
333265 2022-11-08  DXCO3.SA       0      0.257109             0
333379 2022-11-09  DXCO3.SA       0      0.266457             0
333424 2022-11-10  DXCO3.SA       0      0.220990             0
333584 2022-11-11  DXCO3.SA       0      0.408827             0
333651 2022-11-14  DXCO3.SA       0      0.414264             0


In [ ]:
# <-- MUDANÇA: Solução 1 - Geração de Recomendações (baseado em probabilidade)

# --- SESSÃO DE PREVISÃO E RECOMENDAÇÃO ---
print("\n" + "="*50)
print("INICIANDO GERAÇÃO DE RECOMENDAÇÕES FUTURAS")
print(f"Usando modelo treinado para prever a PROBABILIDADE de alta em {PERIOD_HORIZON} dias.")
print(f"Usando os últimos {TIME_STEPS} dias de dados como entrada.")
print("="*50)

if 'model_transformer' in locals() and 'scaler' in locals() and 'features' in locals():
    
    # 1. Preparar os dados mais recentes (do df_merged, antes do dropna)
    # Queremos todas as linhas que tenham os indicadores (features) completos.
    df_predict_input = df_merged.dropna(subset=features)
    
    # 2. Aplicar o Scaler (o mesmo que foi treinado)
    X_predict_scaled_array = scaler.transform(df_predict_input[features])
    
    # Recriar o DataFrame com os dados escalados
    X_predict_scaled = pd.DataFrame(
        X_predict_scaled_array, 
        columns=features, 
        index=df_predict_input.index
    )
    
    # Adicionar de volta o ticker e a data para podermos agrupar
    X_predict_scaled['ticker'] = df_predict_input['ticker']
    X_predict_scaled['datetime'] = df_predict_input['datetime']

    # 3. Montar as sequências de entrada para a previsão
    prediction_sequences = []
    tickers_for_prediction = []
    last_dates = []
    
    unique_tickers = X_predict_scaled['ticker'].unique()
    
    for ticker in unique_tickers:
        # Pegar os dados do ticker e ordenar pela data
        ticker_data = X_predict_scaled[
            X_predict_scaled['ticker'] == ticker
        ].sort_values(by='datetime')
        
        # Verificar se temos dados suficientes para uma sequência
        if len(ticker_data) >= TIME_STEPS:
            # Pegar as últimas TIME_STEPS linhas
            last_sequence = ticker_data[features].iloc[-TIME_STEPS:].values
            
            # Adicionar à lista
            prediction_sequences.append(last_sequence)
            tickers_for_prediction.append(ticker)
            last_dates.append(ticker_data['datetime'].iloc[-1])
        else:
            print(f"Ticker {ticker} ignorado (dados insuficientes: {len(ticker_data)} < {TIME_STEPS})")

    # 4. Fazer as Previsões (Probabilidades)
    if len(prediction_sequences) > 0:
        # Converter a lista de sequências em um array 3D numpy
        X_to_predict = np.array(prediction_sequences)
        print(f"\nGerando previsões para {X_to_predict.shape[0]} tickers...")
        
        # Fazer a previsão (retorna probabilidades)
        future_predictions_proba = model_transformer.predict(X_to_predict).flatten()
        
        # 5. Criar e Rankear o DataFrame de Recomendações
        df_recommendations = pd.DataFrame({
            'ticker': tickers_for_prediction,
            'last_data_date': last_dates,
            'predicted_proba_ALTA': future_predictions_proba
        })
        
        # Ordenar pelas maiores probabilidades de ALTA
        df_recommendations = df_recommendations.sort_values(
            by='predicted_proba_ALTA', 
            ascending=False
        )
        
        print("\n--- TOP 10 RECOMENDAÇÕES (Maior Probabilidade de ALTA) ---")
        print(df_recommendations.head(10))
        
        print("\n--- PIORES 10 (Menor Probabilidade de ALTA) ---")
        print(df_recommendations.tail(10))
        
    else:
        print("Nenhuma sequência válida pôde ser criada para previsão.")
else:
    print("ERRO: 'model_transformer' ou 'scaler' não foram encontrados.")
    print("Certifique-se de que o modelo foi treinado com sucesso antes de rodar esta etapa.")


INICIANDO GERAÇÃO DE RECOMENDAÇÕES FUTURAS
Usando modelo treinado para prever a PROBABILIDADE de alta em 15 dias.
Usando os últimos 30 dias de dados como entrada.

Gerando previsões para 108 tickers...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 

--- TOP 10 RECOMENDAÇÕES (Maior Probabilidade de ALTA) ---
       ticker last_data_date  predicted_proba_ALTA
89   ENJU3.SA     2025-10-28              0.984000
97   ASAI3.SA     2025-10-28              0.975520
75   HAPV3.SA     2025-10-28              0.956848
47   AMAR3.SA     2025-10-28              0.942791
14   TUPY3.SA     2025-10-28              0.931915
106  AZTE3.SA     2025-10-28              0.917421
59   AZZA3.SA     2025-10-28              0.906454
65   ANIM3.SA     2025-10-28              0.906327
26   AZEV3.SA     2025-10-28              0.899902
61   MGLU3.SA     2025-10-28              0.898159

--- PIORES 10 (Menor Probabilidade de ALTA) ---
      ticker last_data_date  predicted_proba_ALTA
86  HBSA3.SA     2025-10-28            

In [ ]:
import time

# 0. Garanta que temos os dados e o modelo
if 'model_transformer' in locals() and 'X_test_seq' in locals() and 'y_test_seq' in locals() and 'features' in locals():
    
    print("Iniciando cálculo de Permutation Feature Importance (usando ROC AUC)...")
    print(f"Testando {len(features)} features...")

    # 1. Calcular o ROC AUC Base (Baseline)
    print("Calculando ROC AUC base...")
    # Usamos as probabilidades (saída bruta do modelo)
    y_pred_base_proba = model_transformer.predict(X_test_seq).flatten() 
    baseline_roc_auc = roc_auc_score(y_test_seq, y_pred_base_proba) # <-- MUDANÇA
    print(f"ROC AUC Base: {baseline_roc_auc:.4f}") # <-- MUDANÇA

    importances = []
    
    # 2. Loop por CADA feature
    for i, feature_name in enumerate(features):
        start_time = time.time()
        
        # Criar uma cópia dos dados de teste
        X_test_permuted = np.copy(X_test_seq)
        
        # 3. Embaralhar (permutar) os valores APENAS da feature 'i'
        values_to_shuffle = X_test_permuted[:, :, i].flatten()
        np.random.shuffle(values_to_shuffle)
        X_test_permuted[:, :, i] = values_to_shuffle.reshape(
            (X_test_seq.shape[0], X_test_seq.shape[1])
        )
        
        # 4. Fazer novas previsões com os dados embaralhados
        y_pred_permuted_proba = model_transformer.predict(X_test_permuted).flatten()
        
        # 5. Calcular o novo ROC AUC
        permuted_roc_auc = roc_auc_score(y_test_seq, y_pred_permuted_proba) # <-- MUDANÇA
        
        # 6. Salvar a QUEDA de importância (na métrica ROC AUC)
        importance_drop = baseline_roc_auc - permuted_roc_auc # <-- MUDANÇA
        importances.append({
            'feature': feature_name,
            'importance_drop': importance_drop
        })
        
        end_time = time.time()
        # <-- MUDANÇA no print para mostrar melhor o drop do AUC
        print(f"  {i+1}/{len(features)}: {feature_name} -> Drop: {importance_drop:+.4f} ({(end_time-start_time):.1f}s)")


    # Converte a lista de resultados em um DataFrame e ordena pela importância
    df_importances = pd.DataFrame(importances).sort_values(by='importance_drop', ascending=False)

    # Exibe os resultados
    print("\n--- TOP 15 Features Mais Importantes (baseado em ROC AUC) ---")
    print(df_importances.head(15).to_markdown(index=False))

    print("\n--- TOP 10 Features Menos Importantes (ou Prejudiciais) ---")
    print(df_importances.tail(10).to_markdown(index=False))
    
    print("\n--- Cálculo de Permutation Importance Concluído ---")
    # df_importances # O DataFrame será a saída da célula
    
else:
    print("ERRO: Variáveis necessárias (modelo, X_test_seq, etc.) não encontradas.")

Iniciando cálculo de Permutation Feature Importance (usando ROC AUC)...
Testando 81 features...
Calculando ROC AUC base...
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
ROC AUC Base: 0.7301
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
  1/81: close -> Drop: -0.0000 (9.4s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
  2/81: high -> Drop: +0.0000 (8.9s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
  3/81: low -> Drop: +0.0000 (9.3s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
  4/81: open -> Drop: -0.0000 (9.3s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
  5/81: volume -> Drop: -0.0000 (9.4s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
  6/81: EMA_9 -> Drop: -0.0000 (9.1s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
  7/81: SMA_21 -> Drop: -0.0000 (8.8s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
  8/81: SMA_50 -> Drop: +0.0000 (8.7s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
  9/81: SMA_200 -> Drop: +0.0000 (8.6s)
2382/2382 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
  10/81: RSI_14 -> Drop: +0.0174 (8.7s)


In [ ]:
# Adicione esta nova célula
# Filtra o DataFrame para pegar apenas features com importância positiva
df_features_v2 = df_importances[df_importances['importance_drop'] > 0]

# Cria a nova lista de features
features_v2 = df_features_v2['feature'].tolist()

print(f"Features originais: {len(features)}")
print(f"Features restantes (v2): {len(features_v2)}")
print("\nFeatures selecionadas (as únicas que ajudam o modelo):")
print(features_v2)

Features originais: 81
Features restantes (v2): 55

Features selecionadas (as únicas que ajudam o modelo):
['RSI_14_men', 'STOCHh_14_3_3_men', 'OBV_men', 'STOCHd_14_3_3_men', 'STOCHk_14_3_3_men', 'DMP_14_men', 'OBV_sem', 'RSI_14', 'ADXR_14_2_men', 'STOCHh_14_3_3_sem', 'OBV', 'ADX_14_men', 'SMA_200_men', 'ADX_14_sem', 'DMN_14_men', 'DMN_14_sem', 'BBB_20_2.0_2.0', 'BBB_20_2.0_2.0_men', 'DMP_14', 'ADXR_14_2_sem', 'BBB_20_2.0_2.0_sem', 'ADX_14', 'BBP_20_2.0_2.0_men', 'MACD_12_26_9_men', 'MACDs_12_26_9_men', 'close_sem', 'ATRr_14_men', 'MACDh_12_26_9_men', 'SMA_200', 'SMA_200_sem', 'BBL_20_2.0_2.0_men', 'STOCHh_14_3_3', 'BBU_20_2.0_2.0', 'BBU_20_2.0_2.0_sem', 'BBM_20_2.0_2.0_sem', 'high', 'STOCHd_14_3_3', 'open_men', 'close_men', 'low', 'BBP_20_2.0_2.0', 'MACDh_12_26_9', 'open_sem', 'SMA_21_sem', 'BBU_20_2.0_2.0_men', 'MACDs_12_26_9_sem', 'EMA_9_men', 'volume_men', 'low_men', 'SMA_50', 'SMA_21_men', 'MACDs_12_26_9', 'high_sem', 'STOCHk_14_3_3', 'BBP_20_2.0_2.0_sem']


In [ ]:
# --- Definição das Features (X) ---
# V2 - Usando a seleção de features da Célula [11] para combater overfitting

# A variável 'features' agora será esta lista otimizada
features = features_v2

if df_final.empty:
    print("ERRO: O DataFrame final está vazio. Rode as células [2] e [3] primeiro.")
else:
    # Garante que as colunas 'y' não estão em 'features'
    colunas_a_excluir = ['datetime', 'ticker', 'close_future', 'y_return', 'y_target']
    features = [col for col in features if col not in colunas_a_excluir]

    print(f"--- USANDO FEATURE SET V2 (OTIMIZADO) ---")
    print(f"Usando {len(features)} features selecionadas:")
    print(features)
    
    # A verificação de Leakage continua importante
    print("\n--- Verificando Vazamento de Dados (Leakage) ---")
    leaky_cols = [col for col in features if col in ['close_future', 'y_return', 'y_target']]

    if len(leaky_cols) > 0:
        print(f"!!! ALERTA DE VAZAMENTO (LEAKAGE) DETECTADO: {leaky_cols} !!!")
    else:
        print(">>> Verificação de leakage OK. Nenhuma coluna-alvo encontrada em X.\n")
    
    # Definir X e y para as células seguintes
    X = df_final[features]
    y = df_final['y_target']

--- USANDO FEATURE SET V2 (OTIMIZADO) ---
Usando 55 features selecionadas:
['RSI_14_men', 'STOCHh_14_3_3_men', 'OBV_men', 'STOCHd_14_3_3_men', 'STOCHk_14_3_3_men', 'DMP_14_men', 'OBV_sem', 'RSI_14', 'ADXR_14_2_men', 'STOCHh_14_3_3_sem', 'OBV', 'ADX_14_men', 'SMA_200_men', 'ADX_14_sem', 'DMN_14_men', 'DMN_14_sem', 'BBB_20_2.0_2.0', 'BBB_20_2.0_2.0_men', 'DMP_14', 'ADXR_14_2_sem', 'BBB_20_2.0_2.0_sem', 'ADX_14', 'BBP_20_2.0_2.0_men', 'MACD_12_26_9_men', 'MACDs_12_26_9_men', 'close_sem', 'ATRr_14_men', 'MACDh_12_26_9_men', 'SMA_200', 'SMA_200_sem', 'BBL_20_2.0_2.0_men', 'STOCHh_14_3_3', 'BBU_20_2.0_2.0', 'BBU_20_2.0_2.0_sem', 'BBM_20_2.0_2.0_sem', 'high', 'STOCHd_14_3_3', 'open_men', 'close_men', 'low', 'BBP_20_2.0_2.0', 'MACDh_12_26_9', 'open_sem', 'SMA_21_sem', 'BBU_20_2.0_2.0_men', 'MACDs_12_26_9_sem', 'EMA_9_men', 'volume_men', 'low_men', 'SMA_50', 'SMA_21_men', 'MACDs_12_26_9', 'high_sem', 'STOCHk_14_3_3', 'BBP_20_2.0_2.0_sem']

--- Verificando Vazamento de Dados (Leakage) ---
>>> Ve

In [ ]:
# A Partir daqui refaz o modelo treinando as features mais relevantes
# <-- MUDANÇA: Solução 2 - Split Treino/Validação/Teste Temporal

if not df_final.empty:
    # 1. Separar Teste (20% finais)
    split_test = int(len(df_final) * 0.8)
    df_train_val = df_final.iloc[:split_test] # 80% para treino+validação
    df_test = df_final.iloc[split_test:]      # 20% para teste final

    # 2. Separar Treino e Validação (dos 80% iniciais)
    split_val = int(len(df_train_val) * 0.8)
    df_train = df_train_val.iloc[:split_val] # 80% do df_train_val
    df_validation = df_train_val.iloc[split_val:] # 20% do df_train_val
    
    # 3. Separar X, y, e tickers para cada set
    X_train = df_train[features]
    y_train = df_train['y_target']
    ticker_train = df_train['ticker']
    
    X_validation = df_validation[features]
    y_validation = df_validation['y_target']
    ticker_validation = df_validation['ticker']
    
    X_test = df_test[features]
    y_test = df_test['y_target']
    ticker_test = df_test['ticker']
    
    print(f"Shape Treino 2D (X_train): {X_train.shape}")
    print(f"Shape Validação 2D (X_validation): {X_validation.shape}")
    print(f"Shape Teste 2D (X_test): {X_test.shape}")
    print(f"Datas de Treino: {df_train['datetime'].min()} a {df_train['datetime'].max()}")
    print(f"Datas de Validação: {df_validation['datetime'].min()} a {df_validation['datetime'].max()}")
    print(f"Datas de Teste: {df_test['datetime'].min()} a {df_test['datetime'].max()}")

    # --- Normalização (Scaling) CORRETA ---
    # 4. Instanciar o Scaler
    scaler = StandardScaler()
    
    # 5. Fit (Ajustar) o scaler APENAS nos dados de TREINO (X_train)
    scaler.fit(X_train)
    
    # 6. Transformar (Aplicar) em TODOS os três sets
    X_train_scaled_array = scaler.transform(X_train)
    X_val_scaled_array = scaler.transform(X_validation)
    X_test_scaled_array = scaler.transform(X_test)
    
    # 7. Criar DataFrames com os dados escalados (para a função de sequência)
    X_train_scaled = pd.DataFrame(X_train_scaled_array, columns=features, index=X_train.index)
    X_val_scaled = pd.DataFrame(X_val_scaled_array, columns=features, index=X_validation.index)
    X_test_scaled = pd.DataFrame(X_test_scaled_array, columns=features, index=X_test.index)
    
    print(f"\nX_train_scaled shape: {X_train_scaled.shape}")
    print(f"X_val_scaled shape: {X_val_scaled.shape}")
    print(f"X_test_scaled shape: {X_test_scaled.shape}")

else:
    print("DataFrame vazio, pulando etapas.")

Shape Treino 2D (X_train): (254270, 55)
Shape Validação 2D (X_validation): (63568, 55)
Shape Teste 2D (X_test): (79460, 55)
Datas de Treino: 2000-01-03 00:00:00 a 2020-01-06 00:00:00
Datas de Validação: 2020-01-06 00:00:00 a 2022-09-23 00:00:00
Datas de Teste: 2022-09-23 00:00:00 a 2025-10-07 00:00:00

X_train_scaled shape: (254270, 55)
X_val_scaled shape: (63568, 55)
X_test_scaled shape: (79460, 55)


In [ ]:

def create_sequences_por_ticker(X_data, y_data, tickers, time_steps):
    """
    Cria sequências de dados 3D (para LSTM) agrupadas por ticker.
    Garante que as sequências não cruzem tickers diferentes.
    """
    all_X_seq, all_y_seq, all_indices = [], [], []
    
    # Usamos os índices originais de df_final para rastrear datas/tickers
    unique_tickers = tickers.unique()
    
    for i, ticker in enumerate(unique_tickers):
        # Filtrar dados para este ticker específico
        ticker_mask = (tickers == ticker)
        X_ticker = X_data[ticker_mask]
        y_ticker = y_data[ticker_mask]
        
        # O índice original é mantido
        ticker_indices = y_ticker.index
        
        # print(f"Processando Ticker: {ticker} ({i+1}/{len(unique_tickers)}) - {len(X_ticker)} amostras") # Debug
        
        # Aplicar a janela deslizante (sliding window) apenas neste ticker
        # Se o ticker tem menos dados que 'time_steps', ele será ignorado
        for j in range(len(X_ticker) - time_steps):
            # Sequência de features (ex: dias 0 a 29)
            seq = X_ticker.iloc[j:(j + time_steps)].values
            
            # Alvo (ex: dia 30)
            target = y_ticker.iloc[j + time_steps]
            
            # Índice do alvo (para rastrear data/ticker depois)
            target_index = ticker_indices[j + time_steps]
            
            all_X_seq.append(seq)
            all_y_seq.append(target)
            all_indices.append(target_index)
            
    return np.array(all_X_seq), np.array(all_y_seq), np.array(all_indices)

# <-- MUDANÇA: Solução 2 - Criar Sequências para os 3 sets
if 'X_train_scaled' in locals():
    
    print("Criando sequências de TREINO...")
    X_train_seq, y_train_seq, seq_indices_train = create_sequences_por_ticker(
        X_train_scaled, 
        y_train, 
        ticker_train, 
        TIME_STEPS
    )
    
    print("Criando sequências de VALIDAÇÃO...")
    X_val_seq, y_val_seq, seq_indices_val = create_sequences_por_ticker(
        X_val_scaled, 
        y_validation, 
        ticker_validation, 
        TIME_STEPS
    )
    
    print("Criando sequências de TESTE...")
    X_test_seq, y_test_seq, seq_indices_test = create_sequences_por_ticker(
        X_test_scaled, 
        y_test, 
        ticker_test, 
        TIME_STEPS
    )
    
    # Esta é a variável que a Célula [9] (Avaliação) usa
    indices_test = seq_indices_test
    
    print(f"\nFormato das Sequências de Treino (X): {X_train_seq.shape}")
    print(f"Formato dos Alvos de Treino (y): {y_train_seq.shape}")

    print(f"\nFormato das Sequências de Validação (X): {X_val_seq.shape}")
    print(f"Formato dos Alvos de Validação (y): {y_val_seq.shape}")
    
    print(f"\nFormato das Sequências de Teste (X): {X_test_seq.shape}")
    print(f"Formato dos Alvos de Teste (y): {y_test_seq.shape}")
else:
    print("Dados de treino/teste não encontrados. Rode as células anteriores.")

Criando sequências de TREINO...
Criando sequências de VALIDAÇÃO...
Criando sequências de TESTE...

Formato das Sequências de Treino (X): (251907, 30, 55)
Formato dos Alvos de Treino (y): (251907,)

Formato das Sequências de Validação (X): (60458, 30, 55)
Formato dos Alvos de Validação (y): (60458,)

Formato das Sequências de Teste (X): (76220, 30, 55)
Formato dos Alvos de Teste (y): (76220,)


In [ ]:
MODEL_FILE = 'transformers_stock_model_30d_ts30_CLASS_weighted_featurev2.keras'

# --- Helper Function: Bloco Transformer Encoder ---
def transformer_encoder_block(inputs, d_model, head_size, num_heads, ff_dim, dropout_rate=0.1):
    """
    Cria um único bloco Transformer Encoder.
    d_model = dimensão da feature (no nosso caso, n_features)
    """
    
    # --- 1. Multi-Head Attention (Self-Attention) ---
    # O modelo "presta atenção" a diferentes partes da sequência de 30 dias
    attn_output = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout_rate
    )(inputs, inputs) # Query=inputs, Key=inputs, Value=inputs (self-attention)
    
    # Conexão Residual e Normalização
    attn_output = layers.Dropout(dropout_rate)(attn_output)
    out1 = layers.LayerNormalization(epsilon=1e-6)(inputs + attn_output)

    # --- 2. Feed Forward Network ---
    # Uma rede neural simples aplicada a cada "dia" (time step)
    ffn_output = layers.Dense(ff_dim, activation="relu")(out1)
    ffn_output = layers.Dense(d_model)(ffn_output) # Projeta de volta para a dimensão original
    
    # Conexão Residual e Normalização
    ffn_output = layers.Dropout(dropout_rate)(ffn_output)
    out2 = layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)
    
    return out2


# --- Função Construtora de Modelo (para KerasTuner) ---
def build_model(hp):
    """
    Função construtora de modelo para o KerasTuner, usando Transformers.
    """
    
    # Pegar as dimensões dos dados de treino
    n_samples, time_steps, n_features = X_train_seq.shape
    d_model = n_features # Dimensão do modelo é o número de features
    
    # --- Input ---
    inputs = layers.Input(shape=(time_steps, d_model), name="Input_Sequence")
    x = inputs
    
    # --- 1. Positional Embedding ---
    # O Transformer puro não sabe a *ordem* dos dias (é permutation-invariant).
    # Precisamos adicionar uma "Positional Embedding" para que ele saiba 
    # qual dia veio antes de qual. Usamos uma Embedding "aprendível".
    
    # Cria uma camada de embedding para as posições (0, 1, ..., 29)
    pos_embedding_layer = layers.Embedding(input_dim=time_steps, output_dim=d_model, name="PositionalEmbedding")
    # Cria as posições (constante)
    positions = tf.range(start=0, limit=time_steps, delta=1)
    # Adiciona o embedding da posição aos dados de entrada
    x = x + pos_embedding_layer(positions)
    
    
    # --- Hiperparâmetros para Tunar ---
    
    # 1. Número de blocos Transformer para empilhar
    hp_num_blocks = hp.Int('num_blocks', min_value=1, max_value=3, step=1)
    
    # 2. Parâmetros da Multi-Head Attention
    hp_num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2) # Ex: 2, 4, 8 cabeças
    hp_head_size = hp.Int('head_size', min_value=32, max_value=128, step=32) # key_dim
    
    # 3. Dimensão da camada Feed-Forward interna
    hp_ff_dim = hp.Int('ff_dim', min_value=d_model, max_value=d_model * 4, step=d_model) # Ex: 128, 256
    
    # 4. Dropout (para regularização)
    hp_dropout = hp.Float('dropout', min_value=0.1, max_value=0.3, step=0.1)
    
    # 5. Learning Rate
    hp_learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=5e-4, sampling='log')
    
    # 6. Unidades da camada Densa (igual ao que você tinha)
    hp_dense_units = hp.Int('dense_units', min_value=32, max_value=64, step=16)
    # ------------------------------------

    # --- 2. Stack de Encoders ---
    # Constrói a arquitetura empilhando os blocos
    for _ in range(hp_num_blocks):
        x = transformer_encoder_block(
            inputs=x,
            d_model=d_model,
            head_size=hp_head_size,
            num_heads=hp_num_heads,
            ff_dim=hp_ff_dim,
            dropout_rate=hp_dropout
        )

    # --- 3. Cabeça de Classificação (Classification Head) ---
    
    # A saída 'x' ainda é uma sequência (Batch, 30, 81).
    # Precisamos agregar tudo em um único vetor por amostra.
    # GlobalAveragePooling1D tira a média dos 30 time steps.
    x = layers.GlobalAveragePooling1D(name="Global_Pooling")(x)
    
    # Camada Densa final para classificação
    x = layers.Dropout(hp_dropout)(x)
    x = layers.Dense(units=hp_dense_units, activation='relu', name="Dense_Classifier")(x)
    
    # Camada final de classificação (sigmoid)
    outputs = layers.Dense(1, activation='sigmoid', name="Output")(x)

    # --- 4. Compilar o Modelo ---
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="Transformer_Tuner_Model")
    
    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='binary_crossentropy',
        metrics=[AUC(name='roc_auc', curve='ROC'), 'accuracy']
    )
    
    return model

# --- Configuração do KerasTuner ---


# Verificar se y_train_seq existe antes de calcular os pesos
if 'y_train_seq' in locals() and y_train_seq.size > 0:
    # Obter as classes únicas (ex: [0, 1])
    classes = np.unique(y_train_seq)
    
    # Calcular os pesos no modo 'balanced'
    # Isso atribui pesos maiores às classes menos frequentes
    weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=y_train_seq
    )
    
    # Criar o dicionário de pesos que o Keras espera
    # Ex: {0: 0.89, 1: 1.14}
    class_weight_dict = dict(zip(classes, weights))
    
    print(f"Pesos de Classe Calculados: {class_weight_dict}")
else:
    print("y_train_seq não encontrado. Pulando cálculo de pesos.")
    class_weight_dict = None # Definir como None se os dados não estiverem prontos


if 'X_train_seq' in locals():
    # Instanciar o Tuner. 
    tuner = kt.Hyperband(
        build_model,
        objective=kt.Objective("val_roc_auc", direction="max"),
        max_epochs=50,
        factor=3,
        directory='keras_tuner_dir',
        project_name='stock_TRANSFORMER_tuning_weighted_rocauc_featuresv2'
    )

    # Callback de EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    print("--- INICIANDO BUSCA POR HIPERPARÂMETROS (TRANSFORMER) ---")
    
    # Iniciar a busca
    tuner.search(
        X_train_seq, y_train_seq,
        epochs=50,
        batch_size=64,
        validation_data=(X_val_seq, y_val_seq),
        callbacks=[early_stopping],
        verbose=1,
        class_weight=class_weight_dict
    )

    print("--- BUSCA CONCLUÍDA ---")

    # 1. Pegar os melhores hiperparâmetros
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"""
    Melhores hiperparâmetros encontrados:
    - num_blocks: {best_hps.get('num_blocks')}
    - num_heads: {best_hps.get('num_heads')}
    - head_size: {best_hps.get('head_size')}
    - ff_dim: {best_hps.get('ff_dim')}
    - dropout: {best_hps.get('dropout'):.2f}
    - dense_units: {best_hps.get('dense_units')}
    - learning_rate: {best_hps.get('learning_rate'):.5f}
    """)

    # -----------------------------------------------------------------
    # SOLUÇÃO: RE-TREINAR O MELHOR MODELO
    # -----------------------------------------------------------------
    print("--- TREINANDO O MODELO FINAL COM OS MELHORES HPs ---")

    # 1. Construir o modelo final usando os melhores HPs
    #    (best_hps já foi pego na etapa anterior)
    model_transformer = build_model(best_hps)

    # 2. (Opcional, mas recomendado) Recupere o número ideal de epochs
    #    se o EarlyStopping parou o melhor "trial" mais cedo.
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
    print("Buscando a melhor época do 'best_trial'...")

    # 1. Pegar o histórico da métrica que seu EarlyStopping monitora (val_loss)
    #    Isso retorna uma lista de objetos: [MetricHistory(epoch=0, value=0.5), MetricHistory(epoch=1, value=0.45), ...]
    val_loss_history_objects = best_trial.metrics.get_history("val_loss")

    # 2. Extrair APENAS os valores numéricos para o numpy
    #    Isso cria uma lista como: [0.5, 0.45, 0.48]
    val_loss_values = [m.value for m in val_loss_history_objects]

    # 3. Encontrar o índice (época) que teve o *menor* valor de loss
    #    Usamos np.argmin() porque "loss" deve ser minimizada.
    best_epoch_index = np.argmin(val_loss_values)

    # 4. Converter o índice (base-0) para o número de épocas (base-1)
    #    Se o índice 0 for o melhor, treinamos por 1 época.
    #    Se o índice 1 for o melhor, treinamos por 2 épocas.
    best_epochs = best_epoch_index + 1

    print(f"Melhor época encontrada (baseada no val_loss): {best_epochs}")

    print(f"Treinando o modelo final por {best_epochs} epochs.")

    # 3. Treinar o modelo final. 
    #    Desta vez, usamos os mesmos dados de treino E validação.
    #    O EarlyStopping é usado para garantir que ele pare no ponto certo.
    history = model_transformer.fit(
        X_train_seq, y_train_seq,
        epochs=best_epochs, # Treina pelo número ideal de épocas
        batch_size=64,
        validation_data=(X_val_seq, y_val_seq),
        callbacks=[early_stopping], # Use o mesmo EarlyStopping
        class_weight=class_weight_dict,
        verbose=1
    )

    print("Modelo final (Transformer) treinado e carregado na variável 'model_transformer'.")
    model_transformer.summary()

    # 4. Salvar o modelo final treinado
    print(f"Salvando o melhor modelo em '{MODEL_FILE}'...")
    model_transformer.save(MODEL_FILE)
    print("Modelo salvo com sucesso.")
    # -----------------------------------------------------------------

else:
    print("AVISO: 'X_train_seq' não foi definido. Pulando hyperparameter tuning.")

Pesos de Classe Calculados: {np.int64(0): np.float64(0.704847898106281), np.int64(1): np.float64(1.7204176967941975)}
Reloading Tuner from keras_tuner_dir\stock_TRANSFORMER_tuning_weighted_rocauc_featuresv2\tuner0.json
--- INICIANDO BUSCA POR HIPERPARÂMETROS (TRANSFORMER) ---
--- BUSCA CONCLUÍDA ---

    Melhores hiperparâmetros encontrados:
    - num_blocks: 1
    - num_heads: 4
    - head_size: 128
    - ff_dim: 34
    - dropout: 0.20
    - dense_units: 64
    - learning_rate: 0.00027
    
--- TREINANDO O MODELO FINAL COM OS MELHORES HPs ---
Treinando o modelo final por 10 epochs.
Epoch 1/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 66s 16ms/step - accuracy: 0.5450 - loss: 0.6722 - roc_auc: 0.5972 - val_accuracy: 0.4922 - val_loss: 0.7240 - val_roc_auc: 0.5804
Epoch 2/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 64s 16ms/step - accuracy: 0.6149 - loss: 0.6287 - roc_auc: 0.6893 - val_accuracy: 0.5997 - val_loss: 0.6545 - val_roc_auc: 0.7043
Epoch 3/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 64s 16ms/step - accuracy

Model: "Transformer_Tuner_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_Sequence      │ (None, 30, 55)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 30, 55)    │          0 │ Input_Sequence[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 30, 55)    │    114,231 │ add_6[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 30, 55)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 30, 55)    │          0 │ add_6[0][0],      │
│                     │                   │            │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 30, 55)    │        110 │ add_7[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 30, 34)    │      1,904 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 30, 55)    │      1,925 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 30, 55)    │          0 │ dense_5[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 30, 55)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 30, 55)    │        110 │ add_8[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Global_Pooling      │ (None, 55)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 55)        │          0 │ Global_Pooling[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Classifier    │ (None, 64)        │      3,584 │ dropout_11[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Output (Dense)      │ (None, 1)         │         65 │ Dense_Classifier… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 365,789 (1.40 MB)

 Trainable params: 121,929 (476.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 243,860 (952.58 KB)

Salvando o melhor modelo em 'transformers_stock_model_30d_ts30_CLASS_weighted_featurev2.keras'...
Modelo salvo com sucesso.


In [ ]:
# <-- MUDANÇA: Solução 1 - Avaliação de Classificação

if 'model_transformer' in locals():
    # 1. Fazer previsões no set de teste (retorna probabilidades)
    y_pred_proba = model_transformer.predict(X_test_seq).flatten()
    
    # 2. Converter probabilidades em classes (ex: 0.5 como threshold)
    y_pred_class = (y_pred_proba > THRESHOLD).astype(int)

    # 3. Buscar as informações originais (data, ticker) usando os índices salvos
    test_info = df_final.loc[indices_test]

    # 4. Criar o DataFrame de resultados
    df_results = pd.DataFrame({
        'date': test_info['datetime'],
        'ticker': test_info['ticker'],
        'y_real': y_test_seq,        # Alvos reais (0 ou 1)
        'y_pred_proba': y_pred_proba,  # Probabilidade prevista (ex: 0.75)
        'y_pred_class': y_pred_class   # Classe prevista (0 ou 1)
    })

    # 5. Avaliação de Métricas de Classificação
    accuracy = accuracy_score(df_results['y_real'], df_results['y_pred_class'])
    print(f"\nAcurácia (Accuracy) no Teste: {accuracy * 100:.2f}%")
    
    # --- CÁLCULO DE ROC AUC ADICIONADO ---
    # (Usa as probabilidades, não as classes)
    roc_auc = roc_auc_score(df_results['y_real'], df_results['y_pred_proba'])
    print(f"ROC AUC no Teste: {roc_auc:.4f}\n")

    print("\nMatriz de Confusão:")
    # (Linhas = Real, Colunas = Previsto)
    print(confusion_matrix(df_results['y_real'], df_results['y_pred_class']))
    
    print("\nRelatório de Classificação:")
    print(classification_report(df_results['y_real'], df_results['y_pred_class'], target_names=['BAIXA (0)', 'ALTA (1)']))

    print("\nAmostra dos Resultados (Transformer):")
    print(df_results.head())

2382/2382 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step

Acurácia (Accuracy) no Teste: 75.02%
ROC AUC no Teste: 0.7628


Matriz de Confusão:
[[47892  7502]
 [11536  9290]]

Relatório de Classificação:
              precision    recall  f1-score   support

   BAIXA (0)       0.81      0.86      0.83     55394
    ALTA (1)       0.55      0.45      0.49     20826

    accuracy                           0.75     76220
   macro avg       0.68      0.66      0.66     76220
weighted avg       0.74      0.75      0.74     76220


Amostra dos Resultados (Transformer):
             date    ticker  y_real  y_pred_proba  y_pred_class
333265 2022-11-08  DXCO3.SA       0      0.060416             0
333379 2022-11-09  DXCO3.SA       0      0.047502             0
333424 2022-11-10  DXCO3.SA       0      0.036787             0
333584 2022-11-11  DXCO3.SA       0      0.166942             0
333651 2022-11-14  DXCO3.SA       0      0.151635             0


In [ ]:
# <-- MUDANÇA: Solução 1 - Geração de Recomendações (baseado em probabilidade)

# --- SESSÃO DE PREVISÃO E RECOMENDAÇÃO ---
print("\n" + "="*50)
print("INICIANDO GERAÇÃO DE RECOMENDAÇÕES FUTURAS")
print(f"Usando modelo treinado para prever a PROBABILIDADE de alta em {PERIOD_HORIZON} dias.")
print(f"Usando os últimos {TIME_STEPS} dias de dados como entrada.")
print("="*50)

if 'model_transformer' in locals() and 'scaler' in locals() and 'features' in locals():
    
    # 1. Preparar os dados mais recentes (do df_merged, antes do dropna)
    # Queremos todas as linhas que tenham os indicadores (features) completos.
    df_predict_input = df_merged.dropna(subset=features)
    
    # 2. Aplicar o Scaler (o mesmo que foi treinado)
    X_predict_scaled_array = scaler.transform(df_predict_input[features])
    
    # Recriar o DataFrame com os dados escalados
    X_predict_scaled = pd.DataFrame(
        X_predict_scaled_array, 
        columns=features, 
        index=df_predict_input.index
    )
    
    # Adicionar de volta o ticker e a data para podermos agrupar
    X_predict_scaled['ticker'] = df_predict_input['ticker']
    X_predict_scaled['datetime'] = df_predict_input['datetime']

    # 3. Montar as sequências de entrada para a previsão
    prediction_sequences = []
    tickers_for_prediction = []
    last_dates = []
    
    unique_tickers = X_predict_scaled['ticker'].unique()
    
    for ticker in unique_tickers:
        # Pegar os dados do ticker e ordenar pela data
        ticker_data = X_predict_scaled[
            X_predict_scaled['ticker'] == ticker
        ].sort_values(by='datetime')
        
        # Verificar se temos dados suficientes para uma sequência
        if len(ticker_data) >= TIME_STEPS:
            # Pegar as últimas TIME_STEPS linhas
            last_sequence = ticker_data[features].iloc[-TIME_STEPS:].values
            
            # Adicionar à lista
            prediction_sequences.append(last_sequence)
            tickers_for_prediction.append(ticker)
            last_dates.append(ticker_data['datetime'].iloc[-1])
        else:
            print(f"Ticker {ticker} ignorado (dados insuficientes: {len(ticker_data)} < {TIME_STEPS})")

    # 4. Fazer as Previsões (Probabilidades)
    if len(prediction_sequences) > 0:
        # Converter a lista de sequências em um array 3D numpy
        X_to_predict = np.array(prediction_sequences)
        print(f"\nGerando previsões para {X_to_predict.shape[0]} tickers...")
        
        # Fazer a previsão (retorna probabilidades)
        future_predictions_proba = model_transformer.predict(X_to_predict).flatten()
        
        # 5. Criar e Rankear o DataFrame de Recomendações
        df_recommendations = pd.DataFrame({
            'ticker': tickers_for_prediction,
            'last_data_date': last_dates,
            'predicted_proba_ALTA': future_predictions_proba
        })
        
        # Ordenar pelas maiores probabilidades de ALTA
        df_recommendations = df_recommendations.sort_values(
            by='predicted_proba_ALTA', 
            ascending=False
        )
        
        print("\n--- TOP 10 RECOMENDAÇÕES (Maior Probabilidade de ALTA) ---")
        print(df_recommendations.head(10))
        
        print("\n--- PIORES 10 (Menor Probabilidade de ALTA) ---")
        print(df_recommendations.tail(10))
        
    else:
        print("Nenhuma sequência válida pôde ser criada para previsão.")
else:
    print("ERRO: 'model_transformer' ou 'scaler' não foram encontrados.")
    print("Certifique-se de que o modelo foi treinado com sucesso antes de rodar esta etapa.")


INICIANDO GERAÇÃO DE RECOMENDAÇÕES FUTURAS
Usando modelo treinado para prever a PROBABILIDADE de alta em 15 dias.
Usando os últimos 30 dias de dados como entrada.

Gerando previsões para 108 tickers...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 

--- TOP 10 RECOMENDAÇÕES (Maior Probabilidade de ALTA) ---
       ticker last_data_date  predicted_proba_ALTA
75   HAPV3.SA     2025-10-28              0.991801
79   LWSA3.SA     2025-10-28              0.990946
103  AURE3.SA     2025-10-28              0.981291
25   PCAR3.SA     2025-10-28              0.978823
61   MGLU3.SA     2025-10-28              0.977187
36   CSAN3.SA     2025-10-28              0.973861
91   BRAV3.SA     2025-10-28              0.971672
56   FLRY3.SA     2025-10-28              0.971361
45   MRVE3.SA     2025-10-28              0.971322
89   ENJU3.SA     2025-10-28              0.967704

--- PIORES 10 (Menor Probabilidade de ALTA) ---
      ticker last_data_date  predicted_proba_ALTA
7   SBSP3.SA     2025-10-28            